In [1]:
import pandas as pd

full = pd.read_csv("../../data/poptrag_lyrics_genres_corpus_filtered_english.csv")
full.info()

<class 'pandas.DataFrame'>
RangeIndex: 111938 entries, 0 to 111937
Data columns (total 21 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   Unnamed: 0                111938 non-null  int64  
 1   track.s.id                111938 non-null  str    
 2   track.s.title             111937 non-null  str    
 3   track.s.firstartist.name  111938 non-null  str    
 4   album.s.title             111938 non-null  str    
 5   album.s.releaseyear       111938 non-null  int64  
 6   track.s.popularity        111938 non-null  int64  
 7   track.language            111938 non-null  str    
 8   full_lyrics               111938 non-null  str    
 9   cat5                      111938 non-null  str    
 10  pmax5                     111938 non-null  float64
 11  nmax5                     111938 non-null  float64
 12  cat12                     111938 non-null  str    
 13  pmax12                    111938 non-null  float64
 14 

In [3]:
from helpers.LyricsClassficationExperiment import LyricsClassificationExperiment

cat5_exp = LyricsClassificationExperiment(
    corpus=full, 
    genrecol="cat5",
    lyricscol="full_lyrics", 
    artistcol="track.s.firstartist.name", 
    output_dir="cat5_mock_experiment",
    test_size=0.2,
    random_state=42, 
    subsample_debug=0.005,
)
print(cat5_exp)

LyricsClassificationExperiment with 5 genres
Train size: 448 samples
Test size: 111 samples
Features not yet computed.
Model not yet trained.
Output directory: cat5_mock_experiment



In [4]:
cat5_exp.compute_fs_ngram_features(min_artists=50, top_n = 100)

✓ Extracted unigrams:
  - Unique: 5,785
  - Shape: (448, 5785)
  - Examples: ['trembling', 'climb', 'apart', 'harborview', 'fuel']
✓ Extracted bigrams:
  - Unique: 31,113
  - Shape: (448, 31113)
  - Examples: ['sailed at', 'blow no', 'after the', 'the barracks', 'from these']
✓ Extracted trigrams:
  - Unique: 48,768
  - Shape: (448, 48768)
  - Examples: ['turn older did', 'cashy rings like', 'all i wish', 'your problem is', "i'll fall or"]
Calculating genre-level TF-IDF for unigrams with genre ...
✓ Calculated TF-IDF for 9,594 genre-ngram pairs
Calculating genre-level TF-IDF for bigrams with genre ...
✓ Calculated TF-IDF for 37,148 genre-ngram pairs
Calculating genre-level TF-IDF for trigrams with genre ...
✓ Calculated TF-IDF for 50,546 genre-ngram pairs
Counting artists per n-gram...
  10% complete
  20% complete
  30% complete
  40% complete
  50% complete
  60% complete
  70% complete
  80% complete
  90% complete
✓ Calculated artist diversity for 5,785 n-grams
Counting artists per

In [6]:
print(cat5_exp)
cat5_exp.train_fixed_parametrer_logistic_regression()

LyricsClassificationExperiment with 5 genres
Train size: 448 samples
Test size: 111 samples
# of features: 135
Feature type: Fell-Spohrleder (2014) N-grams (top 100, min. 50 artists)
Model not yet trained.
Output directory: cat5_mock_experiment

Training pipeline with fixed parameters...


N:\Materialien\Promotion\LyricsGenreRecognition\venv\Lib\site-packages\sklearn\linear_model\_logistic.py:1135: FutureWarning: 'penalty' was deprecated in version 1.8 and will be removed in 1.10. To avoid this warning, leave 'penalty' set to its default value and use 'l1_ratio' or 'C' instead. Use l1_ratio=0 instead of penalty='l2', l1_ratio=1 instead of penalty='l1', and C=np.inf instead of penalty=None.
  warnings.warn(
N:\Materialien\Promotion\LyricsGenreRecognition\venv\Lib\site-packages\sklearn\linear_model\_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


In [10]:
# cat5_exp.show_model_evaluation()
cat5_exp.show_top_coefficients_per_genre()

Top 10 coefficients for genre: ELECTRONIC
but (-1.222)
then (-1.058)
who (-0.876)
i (0.823)
my (-0.792)
out (0.729)
why (0.674)
you (-0.664)
ain't (-0.648)
baby (-0.641)


Top 10 coefficients for genre: HIP HOP
let (-0.868)
said (0.700)
get (0.673)
you know (0.665)
on (0.615)
up (0.599)
man (0.548)
who (0.527)
now (0.503)
they (0.502)


Top 10 coefficients for genre: METAL
need (-1.422)
love (-1.022)
got (-1.011)
up (-0.950)
to (0.832)
this (-0.811)
cause (-0.808)
i'm (-0.742)
no (0.729)
in my (-0.672)


Top 10 coefficients for genre: POP
back (0.973)
got (0.767)
you (0.664)
you know (-0.635)
come (0.628)
on the (0.599)
when you (0.595)
love (0.583)
and (-0.581)
let (0.574)


Top 10 coefficients for genre: ROCK
been (1.191)
you (0.701)
us (-0.638)
in (0.602)
you know (-0.588)
to be (-0.550)
up (0.455)
never (0.440)
no (-0.399)
again (-0.393)


